## Gmail 자동 답장 AI 에이전트 (LangChain + Gemini + Gmail API)

### 프로그램 개요
Gmail에서 읽지 않은 이메일을 자동으로 가져와서 Google Gemini AI로 분류(스팸, 환불요구, 단순문의)하고, 각 카테고리에 맞는 답장 초안을 생성하여 Gmail Drafts에 자동으로 저장하는 AI 에이전트입니다.

### 주요 기능
1. **Gmail API 연동**: OAuth 2.0 인증으로 Gmail 계정에 안전하게 접근
2. **자동 이메일 수집**: INBOX의 읽지 않은 이메일 자동 가져오기
3. **AI 분류**: Google Gemini로 이메일을 SPAM/REFUND/INQUIRY로 분류
4. **자동 답장 생성**: 카테고리별 맞춤형 답장 초안 생성
5. **Draft 자동 저장**: 생성된 답장을 Gmail Drafts에 저장

### 워크플로우
```
Gmail 이메일 가져오기 → Gemini 분류 → 답장 생성 → Gmail Drafts 저장
```

### 목표
1. Google Gemini API 키 설정
2. Gmail API OAuth 인증 구성
3. LangChain 워크플로우 구성 (StateGraph 활용)
4. Gmail과의 완전 자동화 통합

### Step 1. 패키지 설치

In [ ]:
!pip install langgraph langchain-google-genai pydantic google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

### Step 2. Google Gemini API 키 설정

### Step 2-1. Gmail API 설정 및 인증

Gmail API 사용을 위한 준비:
1. [Google Cloud Console](https://console.cloud.google.com/)에서 프로젝트 생성
2. Gmail API 활성화
3. OAuth 2.0 클라이언트 ID 생성 (데스크톱 앱)
4. credentials.json 파일 다운로드하여 프로젝트 폴더에 저장

In [ ]:
import os.path
import pickle
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Gmail API 스코프 설정 (읽기, 쓰기, 수정 권한)
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/gmail.compose',
          'https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    """Gmail API 서비스 객체를 생성하고 인증"""
    creds = None
    
    # token.pickle 파일이 있으면 저장된 인증 정보 로드
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    # 유효한 인증 정보가 없으면 새로 로그인
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        # 인증 정보를 token.pickle에 저장
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    # Gmail API 서비스 생성
    service = build('gmail', 'v1', credentials=creds)
    return service

# Gmail 서비스 초기화
gmail_service = get_gmail_service()
print("Gmail API 인증 완료!")

### Step 2-2. Gmail 이메일 처리 헬퍼 함수

In [ ]:
import base64
from email.mime.text import MIMEText

def get_unread_emails(service, max_results=5):
    """읽지 않은 이메일을 가져옴"""
    try:
        # INBOX의 읽지 않은 이메일 검색
        results = service.users().messages().list(
            userId='me',
            labelIds=['INBOX'],
            q='is:unread',
            maxResults=max_results
        ).execute()
        
        messages = results.get('messages', [])
        
        if not messages:
            print("읽지 않은 이메일이 없습니다.")
            return []
        
        email_list = []
        for message in messages:
            msg = service.users().messages().get(
                userId='me', 
                id=message['id'],
                format='full'
            ).execute()
            
            # 이메일 정보 추출
            headers = msg['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), 'No Subject')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), 'Unknown')
            
            # 이메일 본문 추출
            body = ""
            if 'parts' in msg['payload']:
                for part in msg['payload']['parts']:
                    if part['mimeType'] == 'text/plain':
                        body = base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
                        break
            else:
                if 'body' in msg['payload'] and 'data' in msg['payload']['body']:
                    body = base64.urlsafe_b64decode(msg['payload']['body']['data']).decode('utf-8')
            
            email_list.append({
                'id': message['id'],
                'thread_id': msg['threadId'],
                'subject': subject,
                'sender': sender,
                'body': body
            })
        
        return email_list
    
    except Exception as e:
        print(f"이메일 가져오기 오류: {e}")
        return []

def create_draft_reply(service, thread_id, to, subject, body):
    """Gmail에 답장 초안을 생성"""
    try:
        # 이메일 메시지 생성
        message = MIMEText(body)
        message['to'] = to
        message['subject'] = f"Re: {subject}"
        
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')
        
        # Draft 생성
        draft = service.users().drafts().create(
            userId='me',
            body={
                'message': {
                    'raw': raw_message,
                    'threadId': thread_id
                }
            }
        ).execute()
        
        print(f"✅ Draft 생성 완료: {draft['id']}")
        return draft['id']
    
    except Exception as e:
        print(f"Draft 생성 오류: {e}")
        return None

print("Gmail 헬퍼 함수 정의 완료!")

In [ ]:
import os
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# 1. API 키 설정 (Google Gemini API 키 필요)
os.environ["GOOGLE_API_KEY"] = "your-google-api-key"  # 여기에 Google API Key 입력

# 2. 모델 초기화 (Gemini 모델 사용)
llm = ChatGoogleGenerativeAI(model="gemini-flash-lite-latest", temperature=0)

### Step 3. LangChain 노드 정의

In [ ]:
# 3. 상태(State) 정의: 노드 간에 주고받을 데이터 구조
class EmailState(TypedDict):
    email_id: str           # Gmail 메시지 ID
    thread_id: str          # Gmail 스레드 ID
    sender: str             # 발신자
    subject: str            # 제목
    email_content: str      # 원본 이메일 내용
    category: str           # 분류된 카테고리
    draft: str              # 생성된 답장 초안
    draft_id: str           # Gmail Draft ID

# 4. 노드 함수 정의 (에이전트의 행동)

# [Node 1] Gmail에서 이메일 가져오기
def fetch_email(state: EmailState):
    print("--- 1. Gmail에서 이메일 가져오기 ---")
    # 이미 state에 이메일 정보가 있으면 그대로 반환
    if state.get("email_content"):
        return state
    return state

# [Node 2] 이메일 분류
def classify_email(state: EmailState):
    print("--- 2. 이메일 분류 중 ---")
    prompt = ChatPromptTemplate.from_template(
        "다음 이메일을 분석하여 'SPAM', 'REFUND', 'INQUIRY' 중 하나로 분류해줘. 오직 단어 하나만 출력해.\n\n이메일: {email}"
    )
    chain = prompt | llm
    category = chain.invoke({"email": state["email_content"]}).content.strip()
    print(f"분류 결과: {category}")
    return {"category": category}

# [Node 3] 답장 초안 작성
def draft_reply(state: EmailState):
    print("--- 3. 답장 초안 작성 중 ---")
    category = state["category"]
    email = state["email_content"]
    
    if category == "SPAM":
        print("스팸으로 분류되어 답장을 작성하지 않습니다.")
        return {"draft": "IGNORE"}
    
    prompt = ChatPromptTemplate.from_template(
        "당신은 친절한 고객 지원 AI입니다. 다음 {category} 유형의 이메일에 대한 정중한 답장을 작성해주세요.\n\n원문: {email}"
    )
    chain = prompt | llm
    draft = chain.invoke({"category": category, "email": email}).content
    print(f"답장 초안 생성 완료 (길이: {len(draft)} 글자)")
    return {"draft": draft}

# [Node 4] Gmail에 Draft 저장
def save_to_gmail_draft(state: EmailState):
    print("--- 4. Gmail에 Draft 저장 중 ---")
    
    if state["draft"] == "IGNORE":
        print("스팸 메일이므로 Draft를 생성하지 않습니다.")
        return {"draft_id": "IGNORED"}
    
    draft_id = create_draft_reply(
        service=gmail_service,
        thread_id=state["thread_id"],
        to=state["sender"],
        subject=state["subject"],
        body=state["draft"]
    )
    
    return {"draft_id": draft_id if draft_id else "FAILED"}

print("노드 함수 정의 완료!")

### Step 4. LangChain Workflow 구성 및 컴파일

In [ ]:
# 5. 그래프(Workflow) 구성
workflow = StateGraph(EmailState)

# 노드 추가
workflow.add_node("fetch", fetch_email)
workflow.add_node("classifier", classify_email)
workflow.add_node("drafter", draft_reply)
workflow.add_node("save_draft", save_to_gmail_draft)

# 엣지(흐름) 연결
workflow.set_entry_point("fetch")           # 시작점: 이메일 가져오기
workflow.add_edge("fetch", "classifier")     # 가져오기 -> 분류
workflow.add_edge("classifier", "drafter")   # 분류 -> 답장 작성
workflow.add_edge("drafter", "save_draft")   # 답장 작성 -> Draft 저장
workflow.add_edge("save_draft", END)         # Draft 저장 -> 종료

# 컴파일
app = workflow.compile()
print("워크플로우 컴파일 완료!")

### Step 5. 실행 및 테스트

In [ ]:
# 6. 실행 및 테스트 - Gmail에서 실제 이메일 처리

print("=" * 60)
print("Gmail에서 읽지 않은 이메일을 가져옵니다...")
print("=" * 60)

# Gmail에서 읽지 않은 이메일 가져오기
unread_emails = get_unread_emails(gmail_service, max_results=3)

if not unread_emails:
    print("\n처리할 이메일이 없습니다.")
else:
    print(f"\n총 {len(unread_emails)}개의 읽지 않은 이메일을 찾았습니다.\n")
    
    # 각 이메일에 대해 워크플로우 실행
    for idx, email in enumerate(unread_emails, 1):
        print(f"\n{'=' * 60}")
        print(f"이메일 {idx}/{len(unread_emails)} 처리 중")
        print(f"{'=' * 60}")
        print(f"발신자: {email['sender']}")
        print(f"제목: {email['subject']}")
        print(f"본문 미리보기: {email['body'][:100]}...")
        print()
        
        # 워크플로우 실행
        result = app.invoke({
            "email_id": email['id'],
            "thread_id": email['thread_id'],
            "sender": email['sender'],
            "subject": email['subject'],
            "email_content": email['body']
        })
        
        print(f"\n{'=' * 60}")
        print(f"이메일 {idx} 처리 결과")
        print(f"{'=' * 60}")
        print(f"카테고리: {result['category']}")
        print(f"Draft ID: {result['draft_id']}")
        if result['draft'] != "IGNORE":
            print(f"\n생성된 답장 초안:\n{'-' * 60}\n{result['draft']}\n{'-' * 60}")
        print()

print("\n" + "=" * 60)
print("모든 이메일 처리 완료!")
print("Gmail의 Drafts 폴더를 확인하세요.")
print("=" * 60)